In [1]:
import yaml
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# CARREGAR MODELO
modelo = SentenceTransformer('all-MiniLM-l6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\zoeir\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zoeir\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-l6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# Frases teste
frases = [
    "O gato subiu no telhado",
    "Python é linguagem de programação",
    "O felino escalou a casa",
    "O cachorro escalou a casa"
]

In [4]:
frase_busca = "Um gato no topo da casa"

In [5]:
embeddings = modelo.encode([frase_busca] + frases)

In [6]:
# Calcular similaridades
for i, frase in enumerate(frases):
    sim = cosine_similarity([embeddings[0]], [embeddings[i+1]])[0][0]
    print(f"{sim:.3f} - {frase}")
print(f"\n Maior valor = mais similar a '{frase_busca}'")

0.677 - O gato subiu no telhado
0.203 - Python é linguagem de programação
0.569 - O felino escalou a casa
0.608 - O cachorro escalou a casa

 Maior valor = mais similar a 'Um gato no topo da casa'


In [ ]:
frases = [
    "The cat climbed onto the roof",
    "Python is a programming language",
    "The feline climbed the house",
    "The dog climbed the house."
]

In [8]:
frase_busca = "A cat on top of the house"

In [9]:
# Calcular similaridades em ingles
for i, frase in enumerate(frases):
    sim = cosine_similarity([embeddings[0]], [embeddings[i+1]])[0][0]
    print(f"{sim:.3f} - {frase}")
print(f"\n Maior valor = mais similar a '{frase_busca}'")

0.677 - O gato subiu no telhado
0.203 - Python é linguagem de programação
0.569 - O felino escalou a casa
0.608 - O cachorro escalou a casa

 Maior valor = mais similar a 'A cat on top of the house'


In [10]:
import os
from langchain_chroma import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
import glob
import yaml
import shutil

with open('Config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

In [16]:
#Configuração
gemini_key = config['KEY']
pasta_pdfs = "./manuais" 
CHROMA_PATH = "./chroma_db" #Pasta onde o chromadb sera salvo

# Inicializar
gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                                google_api_key=gemini_key)
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-l6-v2")

In [18]:
# Indexa PDFs no ChromaDB
def indexar_pdfs():
    # Limpar banco existente (opcional)
    if os.path.exists(CHROMA_PATH):
        print("Removendo banco de dados antigo...")
        shutil.rmtree(CHROMA_PATH)
   
    # Carregar PDFs
    documentos = []
    for arquivo in glob.glob("./manuais/*.pdf"):
        print(f"Carregando {os.path.basename(arquivo)}")
        loader = PyPDFLoader(arquivo)
        pages = loader.load()
       
        # Adicionar metadados personalizados
        for page in pages:
            page.metadata['manual'] = os.path.basename(arquivo).replace('.pdf', '')
            documentos.append(page)
   
    if not documentos:
        print("Nenhum PDF encontrado na pasta ./manuais/")
        return
   
    # Criar vector store no ChromaDB
    print(f"Processando {len(documentos)} documentos...")
    vectorstore = Chroma.from_documents(
        documents=documentos,
        embedding=embeddings,
        persist_directory=CHROMA_PATH,
        collection_name="manuais-empresa"
    )
   
    print(f"Indexados {len(documentos)} documentos no ChromaDB!")
    return vectorstore


In [19]:
# Consulta simples
def consultar(pergunta):
    # Verificar se banco existe
    if not os.path.exists(CHROMA_PATH):
        return "Execute 'indexar_pdfs()' primeiro para criar a base de conhecimento!"
   
    # Conectar ao ChromaDB existente
    vectorstore = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings,
        collection_name="manuais-empresa"
    )
   
    # Buscar documentos similares
    results = vectorstore.similarity_search_with_score(pergunta, k=3)
   
    # Verificar se encontrou resultados
    if not results:
        return "Não encontrei informações relevantes nos manuais."
   
    # Montar contexto
    contexto = ""
    for doc, score in results:
        manual = doc.metadata.get('manual', 'Desconhecido')
        texto = doc.page_content[:1000]  # Limitar texto
        # ChromaDB usa distância (quanto menor, melhor), converter para similaridade
        similaridade = 1 - score
        contexto += f"Manual {manual} (relevância: {similaridade:.2f}):\n{texto}\n\n"
   
    # Gerar resposta
    prompt = f"""Com base nos manuais da empresa abaixo, responda de forma clara e prática:

{contexto}

Pergunta: {pergunta}

Resposta baseada nos manuais:"""
   
    return gemini.invoke(prompt).content

In [21]:
print("Indexando os manuais...")
indexar_pdfs()

Indexando os manuais...
Removendo banco de dados antigo...


PermissionError: [WinError 32] O arquivo já está sendo usado por outro processo: './chroma_db\\8bfc1bbf-18b1-4411-b7f3-5abd1da58891\\data_level0.bin'

In [22]:
import time

In [23]:
perguntas = [
    "Como reiniciar o computador?",
    "Qual a senha da impressora?",
    "Como limpar teclado?",
    "Começei hoje no trabalho, qual é o horário do almoço?"
]

for p in perguntas:
    print(f"\n{p}")
    print(f"{consultar(p)}")
    time.sleep(2)


Como reiniciar o computador?
Para reiniciar o computador, siga o procedimento: Menu Iniciar > Energia > Reiniciar (Manual MANUAL_COMPUTADOR).

Qual a senha da impressora?
Os manuais fornecidos não contêm informações sobre a senha da impressora.

Como limpar teclado?
Para limpar o teclado:

1.  **Desconecte** o teclado do computador.
2.  **Vire-o de cabeça para baixo** e balance suavemente para remover detritos soltos.
3.  Use **ar comprimido** entre as teclas para remover poeira e sujeira.
4.  Limpe a superfície com **álcool isopropílico**.
5.  **Nunca use água diretamente** no teclado.

Começei hoje no trabalho, qual é o horário do almoço?
O horário de almoço é das 12h às 13h, de acordo com o manual NORMAS_EMPRESA.
